# Imports, variables, functions

In [1]:
# -- IMPORTS --

import scipy.io
from pyedflib import highlevel
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from itertools import groupby
import csv
import pickle
from scipy.signal import butter, sosfilt, sosfiltfilt, sosfreqz
from scipy.signal import freqz, iirnotch, filtfilt
from sklearn.preprocessing import MinMaxScaler
from sklearn.base import TransformerMixin, BaseEstimator
import random
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from scipy import signal
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import classification_report

In [2]:
# -- ENVIRONMENT VARIABLES --

sample_rate = sampling_rate = 256
sec = 10
len_window = sample_rate*sec
overlap = 5
threshold = 2*sample_rate
sample_rate_downsample = int(0.1*sample_rate)
len_window_downsample = sample_rate_downsample*sec

# Patients
patients = list(range(1,80))
patient_with_issue = [4, 29, 50] # Can't import ECG4, ECG29 and ECG50
for i in patient_with_issue:
    patients.remove(i)

# Load annotation file
annt = scipy.io.loadmat('../raw_data/annotations_2017.mat')

In [3]:
## -- PREPROCESSING FUNCTIONS --

# Highpass filter
def highpass_filter(signals, sampling_rate, hp_frequency = 0.1):
    sos = butter(N = 3, Wn = hp_frequency, btype="highpass",fs=sampling_rate, output="sos")
    filter_hp = sosfiltfilt(sos, signals)
    return filter_hp

# Powerline filter
def notch_filter(signals, sampling_rate, notch_frequency = 50, quality_factor = 30):
    w0 = notch_frequency/(sampling_rate/2)
    b_notch, a_notch = iirnotch(w0, quality_factor)
    filter_notch = filtfilt(b_notch, a_notch, signals, axis = -1)
    return filter_notch

# Create our own scaler
class CustomTranformer(TransformerMixin, BaseEstimator): 
    # BaseEstimator generates the get_params() and set_params() methods that all Pipelines require
    # TransformerMixin creates the fit_transform() method from fit() and transform()
    
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        self.means = X.mean()
        return self
    
    def transform(self, X, y=None):
        norm_features = X - self.means
        return norm_features

# Combination of all filters and Scaler
def filter_signals(signals, sampling_rate, scaler, hp_frequency = 0.1, notch_frequency = 50, quality_factor = 30):
    filter_hp = highpass_filter(signals, sampling_rate)
    filter_notch = notch_filter(filter_hp, sampling_rate, notch_frequency, quality_factor)
    final_signal = scaler.fit_transform(filter_notch)
    return final_signal

In [4]:
## -- LABEL FUNCTIONS --

# Format the EEG 
def eeg_formated(signals, names_ele):
    data_signals = signals.T # transpose the signals from datapoints
    data_signals = pd.DataFrame(data_signals) # create a pandas dataframe
    
    data_signals.columns = names_ele # rename columns
    
    return data_signals

# Format the annotations
def diagnosis(n):
    patient_A=annt["annotat_new"][0][n-1][0]
    patient_B=annt["annotat_new"][0][n-1][1]
    patient_C=annt["annotat_new"][0][n-1][2]
    
    #converting seconds to datapoints

    patient_A=patient_A.tolist()
    patient_B=patient_B.tolist()
    patient_C=patient_C.tolist()
    
    patient_A_dtp=[]
    patient_B_dtp=[]
    patient_C_dtp=[]  
    for elem in patient_A:
        for i in range(sampling_rate):
            patient_A_dtp.append(elem) 
    for elem in patient_B:
        for i in range(sampling_rate):
            patient_B_dtp.append(elem)
        
    for elem in patient_C:
        for i in range(sampling_rate):
            patient_C_dtp.append(elem)
            
    target_=pd.DataFrame({"Diagnosis A":patient_A_dtp,"Diagnosis B":patient_B_dtp,"Diagnosis C":patient_C_dtp})
    
    return target_  

# Create target variables when seizures lasts at least 10
def is_seizure(df):
    
    threshold = sampling_rate*10
    
    df['is_seizure_A'] = df["Diagnosis A"].groupby((df["Diagnosis A"] != df["Diagnosis A"].shift()).cumsum()).transform('size') * df["Diagnosis A"]
    df['is_seizure_A'] = (df['is_seizure_A'] > threshold).astype(int)
    
    df['is_seizure_B'] = df["Diagnosis B"].groupby((df["Diagnosis B"] != df["Diagnosis B"].shift()).cumsum()).transform('size') * df["Diagnosis B"]
    df['is_seizure_B'] = (df['is_seizure_B'] > threshold).astype(int)
    
    df['is_seizure_C'] = df["Diagnosis C"].groupby((df["Diagnosis C"] != df["Diagnosis C"].shift()).cumsum()).transform('size') * df["Diagnosis C"]
    df['is_seizure_C'] = (df['is_seizure_C'] > threshold).astype(int)
    
    return df 

# Create final target
def create_target(df):
    df['is_seizure_target'] = np.where(df['is_seizure_A'] + df['is_seizure_B'] + df['is_seizure_C'] >= 2, 1, 0)
    return df

# Remove useless
def remove_useless_columns(df):
    df.drop(columns=['Diagnosis A', 'Diagnosis B', 'Diagnosis C', 'is_seizure_A', 'is_seizure_B', 'is_seizure_C', 'ECG EKG', 'Resp Effort'], inplace=True)
    return df

# Final function to label
def label_data(path_raw_data, signals_preprocessed, n):
    
    signals, signal_headers, header = highlevel.read_edf(path_raw_data)
    
    names_ele = [signal_headers[iele]['label'] for iele in range(signals.shape[0])] # extract electrode names
    
    eeg_patient = eeg_formated(signals_preprocessed, names_ele) # format the ECG
    eeg_patient.rename(columns={'ECG EKG-REF':'ECG EKG', 'Resp Effort-REF':'Resp Effort'}, inplace=True)
    
    diagnosis_patient = diagnosis(n) # format the diagnosis
    
    data_patient = pd.merge(left=eeg_patient, right=diagnosis_patient, how='left', left_index=True, right_index=True) # merge ecg and diagnosis
    
    is_seizure(data_patient)
    create_target(data_patient)
    remove_useless_columns(data_patient)
    
    return data_patient

In [5]:
## -- FEATURE ENGINEERING --

def flatten(window_df):
    if len(np.unique(window_df.iloc[:,-1])) == 1:
        target = window_df.iloc[0,-1]
    elif np.unique(window_df.iloc[:,-1],return_counts=True)[1][1] >= threshold:
        target = 1
    else:
        target = 0
    t_df = window_df.drop(columns = "target").transpose()
    flatten = pd.DataFrame(np.array(t_df).reshape(1,t_df.shape[0]*t_df.shape[1]))
    flatten["Target"] = target
    return flatten

In [6]:
## -- MAIN FUNCTIONS --

def preprocess_and_label(path_raw_data, scaler, patient_number, Fournier=False):
    
    # Load raw data
    signals, signal_headers, header = highlevel.read_edf(path_raw_data)
    
    # Preprocess data 
    signals_preprocessed = filter_signals(signals, sampling_rate, scaler, hp_frequency = 0.1, notch_frequency = 50, quality_factor = 30)
    
    if Fournier == True:
        signals_preprocessed = pd.DataFrame(np.array([abs(rfft(signals_preprocessed[i])) for i in range(len(signals_preprocessed))]))
        
    # Label data
    df = label_data(path_raw_data, signals_preprocessed, patient_number)
    
    return df

def downsampling(df):
    df_downsample = pd.DataFrame()
    all_df = pd.DataFrame()
    target_col = df.iloc[:,-1]
    num = int(0.1*sample_rate)
    t = 256
    for i, column in enumerate(df.columns[:-1]):
        df_downsample = pd.DataFrame()
        for j in range(0,len(df)-sample_rate,sample_rate):
            x = np.array(df.iloc[j:j+sample_rate,i])
            x_resampled = pd.DataFrame(signal.resample(x, num), index=range(int(j/10),int(j/10)+num))
            df_downsample= pd.concat([df_downsample,x_resampled])
        all_df[column] = np.array(df_downsample).reshape(1,-1)[0]
    target = []
    for t in range(0,len(target_col)-256,sample_rate):
        for n in range(num):
            target.append(target_col[t])
    all_df["target"] = target
    return all_df

def flatten_dataframe(df):
    data = np.array([flatten(df.iloc[i:i+len_window_downsample]) for i in range(0,len(df)-len_window_downsample, overlap*sample_rate_downsample)])
    r=data.shape[0]
    c=data.shape[2]
    data = pd.DataFrame(data.reshape(r,c))
    return data

def concat_dataframes(dictionnary_of_dataframes):
    df = pd.concat([dictionnary_of_dataframes[i] for i in dictionnary_of_dataframes.keys()])
    return df

def create_x_and_y(df):
    X = df.iloc[:,:-1]
    y = df.iloc[:,-1]
    return X,y

def oversampling(X, y): 
    sm = SMOTE(sampling_strategy='minority', random_state=7)
    X, y = sm.fit_resample(X, y)
    return X, y

# Data Preparation

In [8]:
d = {}
for i in patients:
    df_i = preprocess_and_label(f"../raw_data/eeg{i}.edf", CustomTranformer(), i, Fournier=False)
    df_i = downsampling(df_i)
    df_i = flatten_dataframe(df_i)
    d[i] = df_i

In [64]:
# Create train and test for patient 5
df_5_test = d[5].iloc[260:490, :]

df_5_train = d[5].iloc[:260, :]
df_5_train = pd.concat([df_5_train, d[5].iloc[490:,:]])

X_train_5, y_train_5 = create_x_and_y(df_5_train)
X_test_5, y_test_5 = create_x_and_y(df_5_test)

In [67]:
# Create final dataframe
d_without_5 = d.copy()
del d_without_5[5]
df = concat_dataframes(d_without_5)
X, y = create_x_and_y(df)

In [70]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

X_train = pd.concat([X_train, X_train_5])
y_train = pd.concat([y_train, y_train_5])

X_test = pd.concat([X_test, X_test_5])
y_test = pd.concat([y_test, y_test_5])

In [95]:
#X_train.to_csv('../data_modeling/X_train.csv')
#y_train.to_csv('../data_modeling/y_train.csv')
#X_test.to_csv('../data_modeling/X_test.csv')
#y_test.to_csv('../data_modeling/y_test.csv')

In [83]:
# Balance the data
undersample = RandomUnderSampler()

X_train, y_train = undersample.fit_resample(X_train, y_train)

# Modeling default Random Forest

In [137]:
model = RandomForestClassifier(max_depth=10, min_samples_split=10)

## Fit the model

In [138]:
# Fit the model
model.fit(X_train,y_train)

RandomForestClassifier(max_depth=10, min_samples_split=10)

In [139]:
# Train accuracy 
y_pred_train = model.predict(X_train)

train_score = accuracy_score(y_train, y_pred_train)
train_score

0.9170555108608206

## Test on test set

In [140]:
# Test accuracy
y_pred_test = model.predict(X_test)

test_score = classification_report(y_test, y_pred_test)

print(test_score)

              precision    recall  f1-score   support

         0.0       0.95      0.63      0.76     20169
         1.0       0.21      0.76      0.33      2681

    accuracy                           0.64     22850
   macro avg       0.58      0.70      0.55     22850
weighted avg       0.87      0.64      0.71     22850



In [141]:
# Confusion matrix
test_results_df = pd.DataFrame({"actual": y_test,
                           "predicted": y_pred_test})
    
test_confusion_matrix = pd.crosstab(index= test_results_df['actual'],
                               columns = test_results_df['predicted'])
    
test_confusion_matrix

predicted,0.0,1.0
actual,,
0.0,12651,7518
1.0,632,2049


## Test on patient 5: all data (not relevant because part of the data is used for training)

In [142]:
# Preprocess and label new data
path_raw_new_data = "../raw_data/eeg5.edf"
df_new = preprocess_and_label(path_raw_new_data, CustomTranformer(), 5, Fournier=False)

# Downsampling new data
df_new_downsample = downsampling(df_new)

# Flatten new data
df_new_downsample_flat = flatten_dataframe(df_new_downsample)

In [143]:
# Accuracy
y_true = df_new_downsample_flat.iloc[:,-1]
y_pred = model.predict(df_new_downsample_flat.iloc[:,:-1])

new_score = classification_report(y_true, y_pred)
print(new_score)

              precision    recall  f1-score   support

         0.0       0.47      0.75      0.58       132
         1.0       0.94      0.82      0.88       634

    accuracy                           0.81       766
   macro avg       0.70      0.79      0.73       766
weighted avg       0.86      0.81      0.83       766



In [144]:
# Confusion matrix
new_results_df = pd.DataFrame({"actual": y_true,
                           "predicted": y_pred})
    
new_confusion_matrix = pd.crosstab(index= new_results_df['actual'],
                               columns = new_results_df['predicted'])
    
new_confusion_matrix

predicted,0.0,1.0
actual,,
0.0,99,33
1.0,112,522


## Test on patient 5: only the test from patient 5 data

In [150]:
score_specific_window = classification_report(y_test_5, model.predict(X_test_5))
print(score_specific_window)

              precision    recall  f1-score   support

         0.0       0.23      0.69      0.34        48
         1.0       0.83      0.39      0.53       182

    accuracy                           0.45       230
   macro avg       0.53      0.54      0.44       230
weighted avg       0.70      0.45      0.49       230



In [151]:
# Confusion matrix
new_results_df = pd.DataFrame({"actual": y_test_5,
                           "predicted": model.predict(X_test_5)})
    
new_confusion_matrix = pd.crosstab(index= new_results_df['actual'],
                               columns = new_results_df['predicted'])
    
new_confusion_matrix

predicted,0.0,1.0
actual,,
0.0,33,15
1.0,111,71


## Finetune the model: RandomSearch

In [98]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

search = RandomizedSearchCV(model, random_grid, 
                            scoring='recall',
                            n_iter=100,  
                            cv=5, n_jobs=-1)

# Fit data to Random Grid
search.fit(X_train,y_train)

/Users/florencetersier/.pyenv/versions/3.10.6/envs/SeizurePredict/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
# Best score
search.best_score_

In [ ]:
# Best Params
search.best_params_

In [ ]:
# Best estimator
search.best_estimator_

# Modeling finetuned RandomForest

In [ ]:
model_ft = RandomForestClassifier() #change hyperparameters

## Fit the model

In [ ]:
# Fit the model
model_ft.fit(X_train,y_train)

In [ ]:
# Train accuracy 
y_pred_train = model_ft.predict(X_train)

train_score = accuracy_score(y_train, y_pred_train)
train_score

## Test on test set

In [ ]:
# Test accuracy
y_pred_test = model_ft.predict(X_test)

test_score = accuracy_score(y_test, y_pred_test)
test_score

In [ ]:
# Confusion matrix
test_results_df = pd.DataFrame({"actual": y_test,
                           "predicted": y_pred_test})
    
test_confusion_matrix = pd.crosstab(index= test_results_df['actual'],
                               columns = test_results_df['predicted'])
    
test_confusion_matrix

## Test on new data

In [ ]:
# Accuracy
y_true = df_new_downsample_flat.iloc[:,-1]
y_pred = model_ft.predict(df_new_downsample_flat.iloc[:,:-1])

new_score = accuracy_score(y_true, y_pred)
new_score

In [ ]:
# Confusion matrix
new_results_df = pd.DataFrame({"actual": y_true,
                           "predicted": y_pred})
    
new_confusion_matrix = pd.crosstab(index= new_results_df['actual'],
                               columns = new_results_df['predicted'])
    
new_confusion_matrix

## Test on patient 5 specific window

In [ ]:
score_specific_window = classification_report(y_test_5, model_ft.predict(X_test_5))
print(score_specific_window)

In [ ]:
# Confusion matrix
new_results_df = pd.DataFrame({"actual": y_test_5,
                           "predicted": model_ft.predict(X_test_5)})
    
new_confusion_matrix = pd.crosstab(index= new_results_df['actual'],
                               columns = new_results_df['predicted'])
    
new_confusion_matrix